## Review of Strings

In [ ]:
%%file week4.c

#include <stdio.h>
#include <stdlib.h>

int main() {
    char *name;
    strcpy(name, "Alice"); //Bad: name is not a valid address

    name = (char *)malloc(100*sizeof(char));
    strcpy(name, "Alice"); //Ok now, malloc gave valid address

    name[0] = 'a'; //fine

    name = "Alice"; //Ok, but cannot modify name[0]; If we didn't free name, there is memory leak
    name[0] = 'a'; //Could crash

    char name[200] = "Bob";
    strcpy(name, "Alice"); //Fine because there are 200 spaces in name
    //name = "Alice"; //Bad, can't reassign to arrays

    name[0] = 'a'; //OK
}

## Strings in Structs

In [ ]:
%%file week4.c

typedef struct student1 {
    char *name; //Store the addr where the name is stored
    double gpa;
} student1; //Need to allocate name for each student

typedef struct student2 {
    char name[200]; //Store 200 chars
    double gpa;
} student2; //Do not need to allocate name for each student

student1 make_student1(char *name, double gpa) {
    student1 s1;
    s1.name = name;
    s1.gpa = gpa;
    return s1
}

int main() {
    student1 s1 = make_student1("Mike", 3.7);
    printf("Name: %s\n", s1.name);
    //Note: CANNOT DO s1.name[0] = 'm', "Mike" is read-only
}

You could also do: 

In [ ]:
student1 make_student1_new(char *name, double gpa) {
    student1 s1;
    s1.name = (char *)malloc(strlen(name)+1);
    strcpy(s1.name, name);
    s1.gpa = gpa;
    return s1
}

and now it is possible to modify s1.name's contents in the main() function since there is new allocated memory.

For student 2:
<br>
<br>
We don't run into the memory issue since the memory is allocated at char array initialization.

In [ ]:
typedef struct student2 {
    char name[200]; //Store 200 chars
    double gpa;
} student2; //Do not need to allocate name for each student

student2 make_student1(char *name, double gpa) {
    student2 s2;
    strcpy(s2.name, name);
    s2.gpa = gpa;
}

### Example Handout: Basic Pointers
- No variable table like Python, just a memory table

In [ ]:
%%file week4.c

#include <stdio.h>

void f(int *p_a) {
    *p_a = 5; // 2) a = 5
    return 3;// 3) a = 3
}

int main() {
    int a = 15; //1) a = 15
    a = f(&a); //Therefore, a = 3 (local to main)
}

In [ ]:
%%file week4.c

#include <stdio.h>

void f(int *p_a) {
    p_a = malloc(sizeof(int) * 10)
    return 3;
}

int main() {
    int a = 15; 
    a = f(&a);
}

// a = 15 => f(&a) => p_a = 10 new int sizes => a = 3(local to main)
// Therefore a = 3

## Block of Structs

In [ ]:
%%file week4.case

typedef struct student {
    char name[200];
    double gpa;
} student;

typedef struct student2 {
    char *name;
    double gpa;
} student2;

int main() {
    student s1 = {"Mike", 4.8};
    strcpy(s1.name, "Alice");

    //Can now have an array of students:
    student students[200];
    strcpy(students[5].name, "Bob");

    student2 student2s[200];
    // strcpy(student2s[5].name, "Bob"); --> Can't do directly because this index is not a valid addr

    //Can do:
    student2s[5].name = (char *)malloc(200*sizeof(char));
    strcpy(student2s[5].name, "Bob"); //Fine
}


It is also possible to make the block of students using a malloc of students in a function:

In [ ]:
%%file week4.c

#include <stdlib.h>

student2* make_student2_block(int n) {
    student2 *students = (student *)malloc(sizeof(student2)*n);
    for (int i = 0; i < n; i++) {
        students[i].name = (char *)malloc(2000 * sizeof(char));
    }
    return students
}

void destroy_students(student2* students, int n) { //Student2* = pointer to array of student2s
    for (int i = 0; i < n; i++) {
        free(students[i].name); //Free each allocated name memory first
    }
    //Then free the entire array:
    free(students);
}

## realloc

- Can resize blocks of memory using realloc

In [ ]:
%%file week4.c

#include <stdlib.h>

char *str = (char *)malloc(100*sizeof(char));
// Want to make more space
str = (char *)realloc(str, 200 * sizeof(char));

### Error checking
- malloc and realloc might not be able to find the amount of space you need

## exit()
- Unclear what to do otherwise -- trying to access a NULL pointer will lead to a crash without an error msg
- But MS word might just display an error msg, refuse to do what you asked it to (e.g open a huge doc), and continue running
- exit() aims to just crash as gracefully as possible

In [ ]:
%%file week4.c

#include <stdlib.h>

char *block = malloc(100000000);
if (block == NULL) {
    printf("Out of memory\n");
    exit(1); //exit terminates the program, the 1 is sent to operating system
}

## create_str

In [ ]:
%%file week4.c

#include <stdlib.h>

void create_str(char **p_str, int sz) {
   *p_str = (char *)malloc(sz * sizeof(char)); //Init *p_str using the address **p_str
                                               // The pointer to str now has a value
   if (*p_str == NULL){
       printf("Could not create string\n");
       exit(1);
  }
  (*p_str)[0] = '\0'; //Init *p_str with Null terminator
}

int main() {
    char *str;
    /* 
    - Make str be the address of the 1st element of a string
    - HAVE to send in &str to be able to change the value of str
    - If want to change str[0], could just pass str.
    */
    create_str(&str, 100); 
    strcpy(str, "Mike"); 
    return 0;
}

## strcat
- strcat(str1, str2) concatenates str1 and str2, assuming that str1 has enough space to accommodate extra characters from str2
- Will crash if not enough space: it does not check
- Again: C prioritizes efficiency, won't stop you from shooting yourself in the foot


In [ ]:
%%file week4.c

char* strcat(char* str1, char* str2) {
    int i = 0;
    while (str1[i] != NULL) {
        i++;
    }
    //str1[i] here will now be null, can write contents of str2 to the end of str1
    int j = 0;
    while (str2[j] != NULL) {
        str1[i] = str2[j];
        i++;
        j++;
    }
    str1[i] = NULL; //Add back the NULL pointer to the string
    return str1;
}

## ++
int a = 42;
<br>
<br>
int b = a++;
- The value of a++ is the old value of a
- The effect of a++ is to increment a by 1
- The value of b is now 42, the value of a is 43

int c = ++b;
- The value of ++b is the new value of b (i.e. b+1)
- The effect of ++b is to increment b by 1
- The value of b and c is now 42

This can be used to work with pointers

In [ ]:
%%file week4.c

int main() {
    char *str = "abc";
    str++; //Pointer now at str[0] --> str[1] 
    printf("%s\n", str); // This will print "bc"
    printf("%c\n", *str); // This will print 'b'
    printf("%c\n", *str++); //This will print 'b' -- *str was not incremented yet
                            //However, if it was ++*str, then it would be 'c'
    return 0;
}

In [ ]:
%%file week4.c

char* my_strcpy2(char* dest, const char* src) {
    //Keep doing: dest[0]=src[0]
    //Keep incrementing both by 1 simultaneously

    char* dest_start = dest;
    while(*src != '\0') {
        *dest++ = *src++; 
        printf("dest: %ld, src: %ld\n", dest, src);
    }
    *dest = *src; //Copy the line where *src = NULL
    return dest_start;
}

## strcpy and strcat with pointer arithemtic:
- *str2 = *str1 (copies the value from addr str1 to addr str2)
- *str2++ = *str1++ copies values and then increments addresses

In [1]:
%%file week4.c

char* strcpy(char* dest, const char* src) {
    while(*src = NULL) {
        *dest = *src;
        dest++;
        src++;
    }
    *dest = NULL;
    return dest;
}

char* strcpy_increment(char* dest, const char* src) {
    //Keep doing: dest[0]=src[0]
    //Keep incrementing both by 1 simultaneously

    char* dest_start = dest;
    while(*src != '\0') {
        *dest++ = *src++; 
        printf("dest: %ld, src: %ld\n", dest, src);
    }
    *dest = *src; //Copy the line where *src = NULL
    return dest_start;
}

int main() {
    char name1[] = "Mike";
    char name2[1000];
    strcpy_increment(name2, name1);
    printf("Result: %s\n", name2);
}

Overwriting week4.c


In [ ]:
%%file week4.c

char* my_strcat(char *dest, const char *src) {
    /*
    Plan: increment dest while it's not '\0'
    Once there, just do strcpy(dest, src)
    */
    while(*src++); //Continue to increment the pointer
    strcpy_increment(dest, src);
}

## Review: Strings in Structs

In [ ]:
%%file week4.c

#include <stdlib.h>

struct student1 {
    char name[200];
    double gpa;
};

struct student2 {
    char *name;
    double gpa;
};

void create_student1(struct student1 **p_p_s1, const char *name, double gpa) {
    //Allocate memory for [0] element of a student1 array
    *p_p_s1 = (struct student1*)malloc(sizeof(struct student1));
    strcpy((*p_p_s1)->name, name);
    (*p_p_s1)->gpa = gpa;
}

void create_student2(struct student2 **p_p_s1, const char *name, double gpa) {
    *p_p_s1 = (struct student2*)malloc(sizeof(struct student2));
    /*
    - Problem: (*p_p_s1)->name is not initialized (some random number)
               cannot write to the random address (*name does not have allocated memory)
    - Not a problem: (*p_p_s1)->gpa is not initialized (some random number)
        `      we are allowed to write from local int -> struct int
    */
    strcpy((*p_p_s1)->name, name);
    (*p_p_s1)->gpa = gpa;
}

void create_student2_fixed(struct student2 **p_p_s1, const char *name, double gpa) {
    *p_p_s1 = (struct student2*)malloc(sizeof(struct student2));
    
    //Fix: allocate string memory for the names
    (*p_p_s1)->name = (char*)malloc(sizeof(char) * (strlen(name)+1));
    strcpy((*p_p_s1)->name, name);
    (*p_p_s1)->gpa = gpa;
}

int main() {
    struct student1 *p_s1; //p_s1 -> name. struct student1* = struct student1[]
    create_student1(&p_s1);
}

#### Dynamically creating a new array of students:

In [ ]:
%%file week4.c

#include <stdlib.h>

struct student2 {
    char *name;
    double gpa;
};

void create_student2_block(struct student2 **p_p_s1, int n_students) {
    *p_p_s1 = (struct student2*)malloc(sizeof(struct student2) * n_students);
    for (int i = 0; i < n_students; i++) {
        (*p_p_s1)[i].name = (char*)malloc(sizeof(char)*200);
    }
}